<center>
<h1>188.980 Advanced Information Retrieval</h1>
<h2>Exercise 2 - Music Information Retrieval</h2>
</center>

**Group Number:** 12

* **Group members:**
 * **Full Name:** Svitlana Vakulenko
   * *Matrikelnummer:* 01525172
   * *Studienkennzahl:* E 786 881

# Introduction

The aim of this exercise is to get insights into music information retrieval practices. You will follow a content based approach which is based on the idea that the semantically relevant information is provided in the audio itself. Thus the aim is to identify and extract this information in a meaningful representation. 

In [3]:
%load_ext autoreload
%autoreload 2

# plotting functions
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import os              # access to operating system functions
import glob            # reg-ex based file-iterator

# scientific computing
import numpy as np     # numerical computing
import pandas as pd    # powerful data processing library

# audio feature extraction
import librosa
from rp_extract import rp_extract

# machine learning
from sklearn.preprocessing import LabelEncoder, StandardScaler
# use pip install -U scikit-learn to update sklearn if this line throws an error
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC

from utility_functions import show_classification_results, plot_confusion_matrix, show_query_results

# misc
import itertools
import progressbar

# Prepare your data

First, please download the specified data-set, extract it locally to your hardisk and specify the full-path to the extracted directory to the *AUDIO_COLLECTION_PATH* variable below

In [99]:
# GTZAN Genre Collection downloaded from http://marsyasweb.appspot.com/download/data_sets/
# The dataset consists of 1000 audio tracks each 30 seconds long
N_SAMPLES = 1000
# tar -xvf genres.tar into the folder:
AUDIO_COLLECTION_PATH = "./dataset/genres/"

**Prepare file-lists and ground-truth labels**

Define the collection of files to process and the corresponding class labels which is also known as *ground truth*.

* create a list of filenames and a list of corresponding labels
  * e.g. use [glob](https://docs.python.org/2/library/glob.html) to iterate over the class directories in the music collection
  * use the names of the subdirectories as label names
  * iterate through all class directories and get all filenames with the extension '.au'
  * store the filename and its corresponding label in the two lists *filenames* and *labels*
  
Take care that both lists are synchronously aligned and that each label corresponds to the right filename.

In [100]:
# fill these lists with values
filenames = []
labels    = []
# iterate over genre subfolders
for subdir in glob.glob(AUDIO_COLLECTION_PATH+'*'):
    genre = subdir.split('/')[-1]
    
    # get all filenames with the extension '.au'
    genre_files = glob.glob(subdir+'/*.au')
    filenames.extend(genre_files)
    labels.extend([genre]*len(genre_files))
    
    print genre, len(genre_files)
    assert len(filenames) == len(labels)
assert len(filenames) == N_SAMPLES

blues 100
classical 100
country 100
disco 100
hiphop 100
jazz 100
metal 100
pop 100
reggae 100
rock 100


# Music Feature Extraction

*Feature extraction* is a crucial part of content-based approaches. The goal is to transform and reduce the information provided by the digital audio signal into a set of semantically descriptive numbers. A typical CD quality mainstream radio track has an average length of three minutes. This means, that song is digitally described in Pulse-code Modulation (PCM) by 15.9 million numbers (3 [minutes] x 60 [seconds] x 2 [stereo channels] x 44100 [sampling rate]). Using CD-quality 16bit encoding this information requires 30.3MB of memory. Besides music specific reasons, the computational obstacles concerned with processing of huge music collections make raw audio processing a suboptimal solution. Feature design and implementation tries to overcome technological obstacles of digital audio and to extract essential music properties that can be used to analyze, compare or classify music. 

You will use the common music features as descriptors for timbre \cite{logan2000mel}, rhythm \cite{lidy10_ethnic} or general spectral properties \cite{tzanetakis2000marsyas,lartillot2007matlab}.



* iterate over all audio files
    1. load audio data using the [librosa.core.load](http://librosa.github.io/librosa/generated/librosa.core.load.html) function. The feature extractors you will use expect a mono signal as input, so take care to appropriately load or convert the audio data.<br><br>
    2. extract music features<br><br>
        2.1. [Mel-frequency cepstral coefficients (MFCC)](http://librosa.github.io/librosa/generated/librosa.feature.mfcc.html) (librosa): extract 12 coefficients using a FFT-window size of 1024 samples with 50% overlap (this corresponds to a hop-length of 512 samples). To correctly calculate the coefficients the samplerate of the audio file has to be provided.<br>
         2.2. [Pitch-classes / Chroma](http://librosa.github.io/librosa/generated/librosa.feature.chroma_stft.html) (librosa): extract the chroma features using a FFT-window size of 1024 samples with 50% overlap, similar to the MFCCs. Also supply the correct samplerate.<br>
         2.3. The features extracted using the libros-library are provided on a frame-level. They are calculated for each FFT-window and thus the results are lists of feature vectors. The machine learning algorithms you will be using expect a single-vector per instance. Thus, the extracted feature vectors need to be aggregated into a single-vector representation. You will be using the simple approach of just calculating their mean and standard-deviations. Perform the following operations for the *MFCC* and *chroma* features:
      * use the numpy functions *mean* and *std* to calculate the statistical moments of the extracted features along the appropriate axis.
      * use the numpy function *concatenate* to concatenate the statistical moments into a single vector
  
    2.4. *Statistical Spectrum Descriptors* and *Rhythm Patterns*: use the following code to extract the psychoaccoustic feature-sets. The function returns a dictionary of extracted features. The Statistical Spectrum Descriptors are stored using the key 'ssd' and the Rhythm Patterns using the key 'rp'. The features are already single vectors so no post-processing is required.

                rp_features = rp_extract(wave_data, 
                                         samplerate, 
                                         extract_ssd = True, 
                                         extract_rp  = True,
                                         skip_leadin_fadeout = 0, 
                                         verbose     = False)

    3. append the extracted features to the dictionary embedded lists which are provided above.<br><br>
    4. finally iterate over the dictionary keys and convert the lists of extracted feature-vectors into feature-spaces by converting them into numpy arrays using the numpy *asarray* function.

  
The feature extraction will take about 30 minutes

In [76]:
# Fill these lists with values
audio_features = {}
audio_features["mfcc"]   = []
audio_features["chroma"] = []
audio_features["ssd"]    = []
audio_features["rp"]     = []

# progressbar visualization widget to estimate processing time
pbar = progressbar.ProgressBar()

# iterate over all files of the collection
for audio_filename in pbar(filenames):
    
    # 1. load audio
    print audio_filename
    y, sr = librosa.core.load(path=audio_filename, mono=True)
    # 2. extract features
    # - 2.1. mfcc
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=12, n_fft=1024, hop_length=512)
    # - 2.2. chroma
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, n_fft=1024, hop_length=512)
    # -- 2.3. aggregate frame based mfcc and chroma vectors into single feature vectors
    mfcc_means = [np.mean(frame) for frame in mfcc]
    mfcc_stds = [np.std(frame) for frame in mfcc]
    mfcc = np.concatenate([mfcc_means, mfcc_stds])
    
    chroma_means = [np.mean(frame) for frame in chroma]
    chroma_stds = [np.std(frame) for frame in chroma]
    chroma = np.concatenate([chroma_means, chroma_stds])
    # - 2.4. ssd, rp
    rp_features = rp_extract(y, 
                              sr, 
                              extract_ssd = True, 
                              extract_rp  = True,
                              skip_leadin_fadeout = 0, 
                              verbose     = False)
    
    # 3. append to provided lists
    audio_features["mfcc"].append(mfcc)
    audio_features["chroma"].append(chroma)
    audio_features["ssd"].append(rp_features['ssd'])
    audio_features["rp"].append(rp_features['rp'])
    
# # 4. convert lists of vectors to numpy arrays
for key, feature_list in audio_features.items():
    audio_features[key] = np.asarray(feature_list)

./dataset/genres/blues/blues.00000.au
./dataset/genres/blues/blues.00001.au
./dataset/genres/blues/blues.00002.au
./dataset/genres/blues/blues.00003.au
./dataset/genres/blues/blues.00004.au
./dataset/genres/blues/blues.00005.au
./dataset/genres/blues/blues.00006.au
./dataset/genres/blues/blues.00007.au
./dataset/genres/blues/blues.00008.au
./dataset/genres/blues/blues.00009.au


./dataset/genres/blues/blues.00010.au
./dataset/genres/blues/blues.00011.au
./dataset/genres/blues/blues.00012.au
./dataset/genres/blues/blues.00013.au
./dataset/genres/blues/blues.00014.au
./dataset/genres/blues/blues.00015.au
./dataset/genres/blues/blues.00016.au
./dataset/genres/blues/blues.00017.au
./dataset/genres/blues/blues.00018.au
./dataset/genres/blues/blues.00019.au


./dataset/genres/blues/blues.00020.au
./dataset/genres/blues/blues.00021.au
./dataset/genres/blues/blues.00022.au
./dataset/genres/blues/blues.00023.au
./dataset/genres/blues/blues.00024.au
./dataset/genres/blues/blues.00025.au
./dataset/genres/blues/blues.00026.au
./dataset/genres/blues/blues.00027.au
./dataset/genres/blues/blues.00028.au
./dataset/genres/blues/blues.00029.au


./dataset/genres/blues/blues.00030.au
./dataset/genres/blues/blues.00031.au
./dataset/genres/blues/blues.00032.au
./dataset/genres/blues/blues.00033.au
./dataset/genres/blues/blues.00034.au
./dataset/genres/blues/blues.00035.au
./dataset/genres/blues/blues.00036.au
./dataset/genres/blues/blues.00037.au
./dataset/genres/blues/blues.00038.au
./dataset/genres/blues/blues.00039.au


./dataset/genres/blues/blues.00040.au
./dataset/genres/blues/blues.00041.au
./dataset/genres/blues/blues.00042.au
./dataset/genres/blues/blues.00043.au
./dataset/genres/blues/blues.00044.au
./dataset/genres/blues/blues.00045.au
./dataset/genres/blues/blues.00046.au
./dataset/genres/blues/blues.00047.au
./dataset/genres/blues/blues.00048.au
./dataset/genres/blues/blues.00049.au


...

./dataset/genres/rock/rock.00060.au
./dataset/genres/rock/rock.00061.au
./dataset/genres/rock/rock.00062.au
./dataset/genres/rock/rock.00063.au
./dataset/genres/rock/rock.00064.au
./dataset/genres/rock/rock.00065.au
./dataset/genres/rock/rock.00066.au
./dataset/genres/rock/rock.00067.au
./dataset/genres/rock/rock.00068.au
./dataset/genres/rock/rock.00069.au


./dataset/genres/rock/rock.00070.au
./dataset/genres/rock/rock.00071.au
./dataset/genres/rock/rock.00072.au
./dataset/genres/rock/rock.00073.au
./dataset/genres/rock/rock.00074.au
./dataset/genres/rock/rock.00075.au
./dataset/genres/rock/rock.00076.au
./dataset/genres/rock/rock.00077.au
./dataset/genres/rock/rock.00078.au
./dataset/genres/rock/rock.00079.au


./dataset/genres/rock/rock.00080.au
./dataset/genres/rock/rock.00081.au
./dataset/genres/rock/rock.00082.au
./dataset/genres/rock/rock.00083.au
./dataset/genres/rock/rock.00084.au
./dataset/genres/rock/rock.00085.au
./dataset/genres/rock/rock.00086.au
./dataset/genres/rock/rock.00087.au
./dataset/genres/rock/rock.00088.au
./dataset/genres/rock/rock.00089.au


./dataset/genres/rock/rock.00090.au
./dataset/genres/rock/rock.00091.au
./dataset/genres/rock/rock.00092.au
./dataset/genres/rock/rock.00093.au
./dataset/genres/rock/rock.00094.au
./dataset/genres/rock/rock.00095.au
./dataset/genres/rock/rock.00096.au
./dataset/genres/rock/rock.00097.au
./dataset/genres/rock/rock.00098.au
./dataset/genres/rock/rock.00099.au


100% |########################################################################|


# Music Classification

Music Genre Recognition is a well researched Music Information Retrieval task. As for many other audio-based MIR tasks the algorithmic design consists of two parts. First, audio-content descriptors are extracted from the audio signal. In a second step these features are used to train machine learning based models, using popular supervised classifiers including k-nearest neighbors (k-NN), Gaussian mixture models (GMM) or Support Vector Machines (SVM).

## Data-preprocessing

The ground-truth labels are provided using their class names, in our case they are names of music genres, and thus are string values. The applied machine-learning algorithms expect numerical class labels. Thus, you are required to pre-process your data:

 * use sklearn's *LabelEncoder* to convert string- into numeric-labels
 * fit the encoder on the provided genre labels
 * create a list called *encoded_labels* using the *transform* method of the encoder

In [83]:
# docs: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
le = LabelEncoder()
le.fit(labels)
encoded_labels = le.transform(labels)
# 10 genres encoded
print encoded_labels

[0 0 0 0 0 0 0 0 0 0 ..., 9 9 9 9 9 9 9 9 9 9]


## Classification Experiments

Iterate over all feature-sets. For each previously extracted feature perform the following steps:

1. A common way to evaluate the performance of a machine learning approach is to perform a cross-validation. For this exercise you will be using a stratified K-Fold cross-validation. A K-Fold cross-validation splits the data-set into K equally-sized parts and uses K-1 parts to train the classifier and the remaining part to test it. The stratification asserts that the frequency distribution of the class labels within the folds corresponds to the original distribution of the ground-truth assignment. As a first step create a shuffled *StratifiedKFold* cross-validation object which generated 10 *splits* (10-fold cross-validation).<br><br>

2. For each train/test-split apply the following commands:

    2.1. Pre-process the feature-space: audio features are usually abstract content descriptors and have varying value ranges. Many machine learning algorithms expect all features to have the same value ranges. Thus, varying ranges will have a bias towards features with high value ranges. You will use Zero-Mean Unit-Variance Normalization - also referred to as Z-Score normalization or standardization, which subtracts the mean value from each feature and divides it by its variance. An important fact to note here, is that the parameters of the scaler are part of the trained classification model. They have to be assessed from the training-split, because the test-split is supposed to be unseen data. Use the sklearn *StandardScaler* and process the feature-data similar to the *LabelEncoder* by fitting the scaler using only the training-instances. After fitting the scaler transform the feature vectors of the training-split (but do not overwrite the original features!).<br>
    2.2. You will use the Support Vector Machine (SVM) classifier to train a model for automatic genre prediction. The sklearn recently made some efforts to standardize their APIs. Thus, also the SVM classifier adheres to the same pattern. First, create a *LinearSVC* classifier using its default parameters. Then, train the SVM using the *fit* method by supplying the train-split of the feature vecotrs and the ground-truth labels.<br>
    2.3. similar to 2.1. standardize the test-split using the already fitted scaler.<br>
    2.4. use the fitted SVM to predict the scaled test-split feature vectors using its *predict* method.<br>
    2.5. for each fold, store the true and predicted numeric labels.<br><br>
    
3. for each feature-set, store the true and predicted labels of the evaluation back to the provided dictionary.


In [89]:
# fill this dictionary with values
classification_results = {}

pbar = progressbar.ProgressBar()

for feature_name in pbar(audio_features.keys()):
    
    # 1. crossvalidation
    # docs: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
    skf = StratifiedKFold(n_splits=10, shuffle=True)
    
    X = audio_features[feature_name]
    y = encoded_labels
    y_true = [] 
    y_pred = []
    
    # 2. run cross-validation
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # --- TRAIN ---
        
        # 2.1. fit scaler
        # scale training-partition of features
        sc = StandardScaler()
        sc.fit(X_train)
        X_train_scaled = sc.transform(X_train)
        
        # 2.2. create classifier
        clr = LinearSVC()
    
        # fit classifier with scaled training-set
        clr.fit(X_train_scaled, y_train)
        
        # --- TEST ---
        
        # 2.3. scale test-partition of features
        X_test_scaled = sc.transform(X_test)
        
        # 2.4. use fitted classifier to predict labels of test-partition
        y_pred.extend(clr.predict(X_test))
        
        # 2.5. store the true and predicted labels
        y_true.extend(y_test)
        
    # 3. store the classification results back to the dictionary
    classification_results[feature_name] = [y_true, y_pred]
    

100% |########################################################################|


**Plot the classification result table**

If you have supplied the results in the requested format, you should be able to use our provided function to plot the classification results-table.

In [97]:
assert len(classification_results['ssd'][0]) == 1000
assert len(classification_results['ssd'][0]) == 1000
show_classification_results(classification_results)

Featureset,Accuracy,Precision,Recall,F1-Score
rp,0.154,0.123980,0.154,0.090475
ssd,0.138,0.221148,0.138,0.093857
mfcc,0.100,0.010152,0.100,0.018433
chroma,0.100,0.010560,0.100,0.019102


## Feature-set combinations

Music is a complex concept which cannot be described easily by a single property. Music is not just rhythm or a certain timbre/sound. It is more a mixture of all of it. This also applies to music features. Different combinations of features perform better than others, but there is no rule of thumbs which ones. This has to be evaluated according the underlying dataset, the applied classifier, the normalization method, etc. All these parameters affect the performance of the classification model.

For this exercise, extend the previously developed classification approach by a feature combination component. Because the dataset is small, it is feasable to evaluate all possible feature-set combinations. Execute to the following cell to calculate all combinations:

In [ ]:
combinations = []

for i in range(1,len(audio_features.keys())):
    combinations.extend(itertools.combinations(audio_features.keys(), i))

Combining features is also referred to as fusion and two common approaches, early fusion and late fusion, exist. You will be using an early fusion approach to extend the classification experiment. To do so, either copy and paste the code of the evaluation-loop into a new cell, or adapt the code of the existing cell:

* in the for loop, change the iterable from *audio_features.keys()* to *combinations*. You are also adviced to change the variable name *feature_name* to reflect the circumstance that the variable contains now a list of feature names.
* implement the early fusion approach. This is accomplished by appending one feature vector to the other to create a combined feature vector. This can be solved by combining the feature-spaces. Use the numpy function *concatenate* to concatenate the feature combinations along feature axis. This can be done before the cross-validation loop or within. 
* be sure to reset the *classification_results* dictionary before running the evaluation again.

In [90]:
print combinations

NameError: name 'combinations' is not defined

**Plot the classification result table and confusion matrix**

If you have supplied the results in the requested format, you should be able to use our provided function to plot the classification results-table and the confusion matrix. A confusion matrix is a convenient tool to assess class-level model performance. It shows which classes are mutually confused. This can help to tune the parameters of the model or to choose different features/combinations.

To estimate the performance of the evaluated models:
* plot four result tables and confusion matrices including the top-performing result.
* write a paragraph where you summarize the conclusions of the classification experiments
  * which feature-set/feature-set-combinations perform best?
  * for which classes are they performing best?
  * which classes are confused most?
  * what does this tell about the audio features?

In [ ]:
show_classification_results(classification_results)

In [ ]:
plot_confusion_matrix(classification_results["ssd"], l_encoder)

### Conclusions

Write your conclusions here...

# Similarity Retrieval / Query by example

The second part of the assignment focuses on retrieving similar songs to a given example. Content based music similarity can also be defined upon the numeric feature-space. Thus, you will use the extracted music features from the classification experiments, but now you will implement a late fusion approach. The problem with early fusion is that results will skew towards features-sets with higher dimensionality. Late fusion represents a convenient method to overcome this problem by combining the result-spaces instead of the feature-spaces. For each feature first the similarities are calculated separately and then aggregated into a final estimation. Thus, every feature has an equal influence on the final result.

**Normalization**

Just like for the classification experiments the feature-spaces first need to be normalized. 

* iterate over all feature-sets
* use the *StandardScaler* as above, but because there are no train/test-splits, fit the scaler using all instances of a feature-set, and finally perform the transformation
* you can overwrite the extracted *audio_features*; their original values are not required further.

In [ ]:
# add your code here


**Implement a query-by-example system:**

* define a *query_id* variable. This represents the index into the feature-space. The feature-vector with the corresponding id will be used to find similar songs.
* define a list of feature-set names. This is equivalent to the feature combinations you have implemented for the classification experiments.
* define a corresponding list of feature-weights. Usually weights scale from 0.0 to 1.0 and represent the influence of a feature-set on the final result.
* for each feature-set of the specified feature-set combination:
  * calculate the Eucledian-distance from the query-feature-vector to every other vector of a feature-set:

$$
\sum \sqrt{(audio\_features[feature\_name] - audio\_features[feature\_name][query\_id])^{2}}
$$

  * this will result in a distance value for each feature-vector representing the dis-similarity to the query-vector.
  * due to variances in dimensionality and value-ranges these distances vary between the feature-sets. To normalize the impact of a distinct feature-set in a late-fusion approach, you have to normalize each resulting distance-list by dividing it by its maximum value.
  * multiply each normalized distance-list with its corresponding feature-weight
  * calculate the sum of the weighted distance-lists
  * finally, use apply numpy function *argsort* on the final results to get the indexes to the feature-vectors ranked by their ascending summed distances. Thus, the top results have the lowest distance/dis-similarity to the query-vector and are expected to sound similar to the query-song.
  * use our provided method to display the retrieval results:
  
        show_query_results(filenames, labels, ranked_index)
      
The displayed result-table contains HTML5 audio player tags. Unfortunately, most Internet browsers have a security-lock to prevent Web-pages from referencing or accessing local files. 

* To enable the playback of the audio-files, open a command-prompt and change to the root directory of the provided dataset (*AUDIO_COLLECTION_PATH*). 
* start the Python simple web-server on port 5555. This will create a simple web-server which handles requests to the local files:

        > python -m SimpleHTTPServer 5555
        

In [ ]:
# add your code here


* Experiment with different feature-set combinations and feature-weights using different query-songs.
* show two good and two bad performing examples using the *show_query_results* function
* write a paragraph where you summarize the conclusions of the query-by-example experiments
  * which feature-sets are performing best and why?
  * listening to the results, what is the perceived quality?
    * in which terms do the top-ranked results sound similar
    * are there songs that do not fit at all?
    * do they at least share some acoustic similarity with the query-song (e.g. rhythm, instrumentation, pitch, etc.)?
  * compare the query-by-example precision with the classification confusion-matrix for the corresponding feature-set-combination. Are the top confusions also predominant mismatches within the top-ranked retrieval results?

### Conclusions

Write your conclusions here...